In [3]:
import os
import re

In [89]:
def read_dir(folder_path):
    """
    读取文件夹下所有符合条件的文件
    :param folder_path: 文件夹路径
    :return: 经过排序后的文件列表
    """
    if not os.path.exists(folder_path):
        print('Folder does not exist')
        return 
    file_list = []
    
    # 获取文件夹下所有符合条件的文件
    for filename in os.listdir(folder_path):
        if filename.endswith(".sdx") and "_" in filename and filename.split("_")[1][:-4].isdigit():
            file_list.append(filename)
    file_list.sort(key=lambda x: int(x.split("_")[1][:-4].split(".")[0]))
    return file_list

def trim_files(src_path,target_path):
    """
    将把文件夹下所有文本按照分号分割成句子，去除空格和换行符，写入新文件
    :param folder_path: 文件夹路径
    :param target_path: 目标保存文件夹路径
    :return: None
    """
    if not os.path.exists(src_path):
        print('Folder does not exist')
        return
    if not os.path.exists(target_path):
        os.makedirs(target_path)

    file_list = read_dir(src_path)
    for filename in file_list:
        with open(src_path + filename, 'r') as f, open(target_path+filename, 'w') as out_f:
            sentence = ""
            index = 0
            for line in f:
                index = index + 1
                line = line.strip()
                if(index <2):
                  out_f.write(line + '\n')
                  continue
                else:
                    if re.search(r'[;]$', line):
                        sentence += line
                        out_f.write(sentence + '\n')
                        sentence = ""
                    else:
                        sentence += line
        if sentence:
            out_f.write(sentence + '\n')

# 判断文件中的语句是什么类型的语句
# 1. 函数调用
# 2. 对象调用
# 3 注释
# 4. 空行
# 5. 其他 ,声明，赋值




def is_comment(line):
    return line.startswith('//')

def is_empty(line):
    return line == '\n'


def is_func_call(line):
    pattern = r'\w+\(.*\);'  
    if re.match(pattern, line):
        return True
    else:
        return False

def is_object_func_call(line):
    pattern = r'\w+->\w+\(.*\);'
    if re.match(pattern, line):
        return True
    else:
        return False


def is_other(line):
    if is_comment(line) or is_empty(line) or is_function_call(line) or is_object_function_call(line):
        return False
    else:
        return True


###################################
### 解析一个语句，返回一个词列表，判断这些词的属性

def is_decimal_or_hex(string):
  """
  判断是否是十进制或十六进制
  :param string
  :return:bool
  """
  string = string.strip()
  if not string:
    return False

  try: 
    int(string, 10)
    return True
  except ValueError:
    pass

  try:
    int(string, 16)
    return True
  except ValueError:
    pass

  return bool(re.match(r'^[0-9a-fA-F]+$', string))


def is_macro(string):
  """
  判断是否是宏定义
  :param string
  :return:bool
  """
  return bool(re.fullmatch(r'[A-Z0-9_]+', string))


def is_valid_variable(string):
    pattern = r'^[a-zA-Z][a-zA-Z0-9_]*$'
    return re.match(pattern, string) and any(c.islower() for c in string)

### 如果是一个声明，那么只需要提取等号左边的变量，
# 如果是函数，函数的种类不同，提取的变量也不同


# 解析一个函数调用，返回函数名和参数列表
def parse_func_call(func_call):
    func_name = func_call.split('(')[0]
    func_args = func_call.split('(')[1].split(')')[0].split(',')
    func_args = [arg.strip() for arg in func_args]
    return func_name, func_args
#解析一个对象函数调用，返回对象名、函数名和参数列表
def parse_object_func_call(statement):
    pattern = r'(\w+)\s*->\s*(\w+)\s*\((.*?)\);'
    match = re.match(pattern, statement)
    
    if match:
        object_name = match.group(1)
        function_name = match.group(2)
        arguments = match.group(3)
        argument_list = [arg.strip() for arg in arguments.split(',')] if arguments else []
        return object_name, function_name, argument_list
    else:
        return None

# 解析一个声明语句，返回声明变量类型，声明变量，赋值
def parse_statement(string):
  pattern = r'(\w+\*?)\s*([\w+]+\[.*?]*)\s*=\s*(.*?);'
  match = re.match(pattern, string)
  if match:
    a = match.group(1)
    b = match.group(2) if match.group(2) else ""
    c = match.group(3)
    return a,b,c
  else:
    return None



In [3]:
src_path = "D:\\test\\script\\3DMark_CloudGate-orgSize-FixFPS0.5-GT2\\vector\\"
target_path = "output\\"
trim_files(src_path,target_path)


Folder does not exist


In [85]:
# 查看各种语句的几个例子
folder_path = '.\output\\'
file_list = read_dir(folder_path)
for filename in file_list:
        with open(folder_path + filename, 'r') as f:
            for line in f:
                if is_other(line):

axCreateWindow(x = 0, y = 0, w = 1280, h = 720, depth = 0, xvis = NULL, cfg = NULL)=win_1;



In [95]:
# 读取一个文件，解析每一行，判断每一行语句的种类，然后进行处理
# 1. 读取文件
# 2. 解析每一行
# 3. 判断语句种类
# 4. 处理语句

with open(r"output\3DMarkICFWorkload_58.sdx", "r") as f:
    for line in f:
        if is_comment(line):
            print(line)
        elif is_empty(line):
            print(line)
        elif is_object_func_call(line):
            print(parse_object_func_call(line))
        elif is_func_call(line):
            pass
        elif is_other(line):
            pass
        

//app: C:\ProgramData\UL\3DMark\chops\dlc\cloud-gate-test\bin\x64\3DMarkICFWorkload.exe

('pSwap_0', 'GetBuffer', ['Buffer = 0', 'riid = IID_ID3D11Texture2D', 'ppSurface = pSwap_0_buf_0'])
('pCtx_0', 'ClearRenderTargetView', ['pRenderTargetView = pRTV_1', 'ColorRGBA = color_194'])
('pCtx_0', 'DiscardView', ['pResourceView = pRTV_22'])
('pCtx_0', 'Map', ['pResource = pBuf_13', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9860'])
('pCtx_0', 'Unmap', ['pResource = pBuf_13', 'Subresource = 0'])
('pCtx_0', 'Map', ['pResource = pBuf_15', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9861'])
('pCtx_0', 'Unmap', ['pResource = pBuf_15', 'Subresource = 0'])
('pCtx_0', 'Map', ['pResource = pBuf_16', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9862'])
('pCtx_0', 'Unmap', ['pResource = pBuf_16', 'Subresourc